In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9MB 111kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 16.0MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595702 sha256=9a44682be80edb293fc128de1f4f2ee447db5f2c0f6a51aa0cb6507a6fc6ef7c
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 14.7MB/s 
     |████████████████████████████████| 757kB 13.9MB/s 
     |████████████████████████████████| 3.0MB 37.6MB/s 
     |████████████████████████████████| 901kB 49.7MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-x2k1mkg4
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-x2k1mkg4
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12718 sha256=7c0291ceb5c32cf04bb075fde39f4492d5000d8d1cc227eaf34dd8e73b8e5fdb
  Stored in directory: /tmp/pip-ephem-wheel-cache-uje6p_t0/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
!wget "https://drive.google.com/uc?export=download&id=1fhEBkjvl0g7EHYPjN-r8wefEUi_pZ7RZ" -O datatest.tsv
!wget "https://drive.google.com/uc?export=download&id=1rmjklQfdEPaDgpQu1UKi64iagytxrC92" -O datatrain.tsv

--2021-06-28 21:57:40--  https://drive.google.com/uc?export=download&id=1fhEBkjvl0g7EHYPjN-r8wefEUi_pZ7RZ
Resolving drive.google.com (drive.google.com)... 108.177.119.139, 108.177.119.113, 108.177.119.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.119.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-8g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qo4sc7q35ap6k4734vptpojgm269ti8f/1624917450000/16667090296722432029/*/1fhEBkjvl0g7EHYPjN-r8wefEUi_pZ7RZ?e=download [following]
--2021-06-28 21:57:41--  https://doc-0k-8g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qo4sc7q35ap6k4734vptpojgm269ti8f/1624917450000/16667090296722432029/*/1fhEBkjvl0g7EHYPjN-r8wefEUi_pZ7RZ?e=download
Resolving doc-0k-8g-docs.googleusercontent.com (doc-0k-8g-docs.googleusercontent.com)... 172.217.218.132, 2a00:1450:4013:c08::84
Connecting to doc-0k-8g-docs.googleusercontent.com (d

In [ ]:
dataset_train = nlp.data.TSVDataset("datatrain.tsv")

In [ ]:

dataset_test = nlp.data.TSVDataset("datatest.tsv")

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(int(float(i[label_idx]))) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 8
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
print(dataset_train[:5])

[['나이 들어서 걸리면 치사 율도 높을껀데 전부 자기 들 자초 일이 의료 진 죽어나는거지', '0'], ['축하 합니다', '1'], ['저 것 들 한 곳 몰아 넣고 지 들 끼리 감염 되서 물 뜯 고함 다 디지길', '0'], ['몇 일 전 부터 우한 중국 입국 시키더니 우한 폐렴 확 진자 급증 이 거 아무래도 연관 성 의심 안 할수가 없다', '0'], ['왜 또 교회 몰려서 이런 난리 피우는 건지 집단 장소 가지말라고 그렇게 당부 메세지 받았을텐데 정말 이번 말 되면 참 많은 사망자 발생 할것 같다 ㅠㅠ', '0']]


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
#print(data_train[0])

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    print("epoch {} test loss {}".format(e+1,loss.data.cpu().numpy() ))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7749324440956116 train acc 0.359375
epoch 1 batch id 201 loss 0.5158419013023376 train acc 0.8232276119402985

epoch 1 train acc 0.8477496483825597


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.9021120124549328


...


epoch 6 test acc 0.9437581940347427


epoch 7 batch id 1 loss 0.11171405762434006 train acc 0.96875
epoch 7 batch id 201 loss 0.18566884100437164 train acc 0.908193407960199

epoch 7 train acc 0.9203059071729958



epoch 7 test acc 0.9401630612913798


epoch 8 batch id 1 loss 0.10573995858430862 train acc 0.96875
epoch 8 batch id 201 loss 0.17050890624523163 train acc 0.9125466417910447

epoch 8 train acc 0.9223057313642757



epoch 8 test acc 0.9443112913798755


In [ ]:

import pandas as pd

def getSentimentValue(comment, tok, max_len, batch_size, device):
  commnetslist = []
  emo_list = []
  for c in comment:
    commnetslist.append( [c, 5] )
  pdData = pd.DataFrame( commnetslist, columns = [['댓글', '감성']] )
  pdData = pdData.values
  test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):#enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length 
    out = model(token_ids, valid_length, segment_ids)
    #print("out one:",out)
  
    for e in out:
      #print(e)
      if e[0]>e[1]: # 부정
        value = 0
      else: #긍정
        value = 1
      emo_list.append(value)

  return emo_list


In [ ]:
from google.colab import drive
import json

tlist = [] # database table name list
for i in range(1,6):
    t = "daum"
    tlist.append(t+"after"+str(i))
    tlist.append(t+"before"+str(i))
for i in range(1,6):
    t = "naver"
    tlist.append(t+"after"+str(i))
    tlist.append(t+"before"+str(i))

drive.mount("/content/drive")
path = "/content/drive/My Drive/ColabNotebooks/jsonComment/" # comment  json file directory path
for tablename in tlist:
    
    with open(path+tablename+'-dict.json', encoding="utf-8") as json_file:
        data = json.load(json_file) # load json file
    
    for date in data.keys(): 
        #print(">>",date,"/",tablename)
        
        day_article = data[date]
        for article in day_article.keys():
            #if len(day_article[article][0])>100: print("카운팅:",len(day_article[article][0]))
            comments = day_article[article][0]
            if len(comments)==1: continue
            
            comments.remove("")
            
            emotion = getSentimentValue(comments, tok, max_len, batch_size, device)
            
            data[date][article][-1].extend(emotion)
            #data[date][article][-1].append(emotion) # dictionary value append sentiment
    print(tablename, "--comment save")
    result_path = "/content/drive/My Drive/ColabNotebooks/predictComment/" #  result save json path
    #drive.mount("/Colab Notebooks/predictComment")
    with open(result_path+"finish-"+tablename+"-dict.json", "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, indent="\t",ensure_ascii = False)



Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


daumafter1 --comment save
daumbefore1 --comment save
daumafter2 --comment save
daumbefore2 --comment save
daumafter3 --comment save
daumbefore3 --comment save
daumafter4 --comment save
daumbefore4 --comment save
daumafter5 --comment save
daumbefore5 --comment save
naverafter1 --comment save
naverbefore1 --comment save
naverafter2 --comment save
naverbefore2 --comment save
naverafter3 --comment save
naverbefore3 --comment save
naverafter4 --comment save
naverbefore4 --comment save
naverafter5 --comment save
naverbefore5 --comment save
